In [1]:
import pandas as pd
import numpy as np
import requests

 Прочитаем файл JSON, который сохранили в ex02

In [2]:
df = pd.read_json('../data/auto.json')
df

CarNumber  Refund    Fines    Make    Model
0    Y163O8161RUS       2   3200.0    Ford    Focus
1     E432XX77RUS       1   6500.0  Toyota    Camry
2     7184TT36RUS       1   2100.0    Ford    Focus
3    X582HE161RUS       2   2000.0    Ford    Focus
4    92918M178RUS       1   5700.0    Ford    Focus
..            ...     ...      ...     ...      ...
720  Y163O8161RUS       2   1600.0    Ford    Focus
721  M0309X197RUS       1  22300.0    Ford    Focus
722  O673E8197RUS       2    600.0    Ford    Focus
723  8610T8154RUS       1   2000.0    Ford    Focus
724  H419XE197RUS       2   8594.6  Toyota  Corolla

[725 rows x 5 columns]

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725 entries, 0 to 724
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CarNumber  725 non-null    object 
 1   Refund     725 non-null    int64  
 2   Fines      725 non-null    float64
 3   Make       725 non-null    object 
 4   Model      716 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 28.4+ KB


◦ один из столбцов имеет тип float, поэтому определим его формат в
pandas с помощью pd.options.display.float_format: значения с плавающей запятой должны отображаться с двумя десятичными знаками

◦ в модели отсутствуют значения, ничего с ними не делаем

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

Добавим новые данные (выдуманные штрафы). Закрепленные значения - номер, марка, модель. Возврат и штрафы рандомные из имеющихся.


In [4]:
sample = df.sample(n=200, random_state=21).reset_index(drop=True)
sample

CarNumber  Refund     Fines        Make   Model
0    M0299X197RUS       2 19,200.00        Ford   Focus
1    83298C154RUS       2  8,594.60        Ford   Focus
2    H957HY161RUS       1  2,000.00        Ford   Focus
3     T941CC96RUS       1  2,000.00        Ford   Focus
4    H966HY161RUS       1    500.00        Ford   Focus
..            ...     ...       ...         ...     ...
195  8182XX154RUS       1    200.00        Ford   Focus
196   X796TH96RUS       1    500.00        Ford   Focus
197  T011MY163RUS       2  4,000.00        Ford   Focus
198   T341CC96RUS       2  1,000.00  Volkswagen  Passat
199   T119CT96RUS       1  2,000.00        Ford   Focus

[200 rows x 5 columns]

Выберем 200 случайных значений для возвратов и штрафов и заменим ими столбцы в нашей выборке

In [5]:
sample_for_Refund = df.sample(n=200, random_state=42).reset_index(drop=True)
sample_for_Refund['Refund']

0      1
1      2
2      2
3      1
4      1
      ..
195    1
196    1
197    2
198    2
199    2
Name: Refund, Length: 200, dtype: int64

In [6]:
sample_for_Fines = df.sample(n=200, random_state=30).reset_index(drop=True)
sample_for_Fines['Fines']

0     8,300.00
1     8,594.60
2     1,300.00
3     2,400.00
4     2,400.00
        ...   
195   9,500.00
196   2,000.00
197   4,400.00
198     800.00
199   1,400.00
Name: Fines, Length: 200, dtype: float64

In [7]:
sample['Fines'] = sample_for_Fines['Fines']
sample['Refund'] = sample_for_Refund['Refund']
sample

CarNumber  Refund    Fines        Make   Model
0    M0299X197RUS       1 8,300.00        Ford   Focus
1    83298C154RUS       2 8,594.60        Ford   Focus
2    H957HY161RUS       2 1,300.00        Ford   Focus
3     T941CC96RUS       1 2,400.00        Ford   Focus
4    H966HY161RUS       1 2,400.00        Ford   Focus
..            ...     ...      ...         ...     ...
195  8182XX154RUS       1 9,500.00        Ford   Focus
196   X796TH96RUS       1 2,000.00        Ford   Focus
197  T011MY163RUS       2 4,400.00        Ford   Focus
198   T341CC96RUS       2   800.00  Volkswagen  Passat
199   T119CT96RUS       2 1,400.00        Ford   Focus

[200 rows x 5 columns]

Добавим эти строкм в основной датафрейм

In [8]:
concat_rows = pd.concat([df, sample])
concat_rows

CarNumber  Refund    Fines        Make   Model
0    Y163O8161RUS       2 3,200.00        Ford   Focus
1     E432XX77RUS       1 6,500.00      Toyota   Camry
2     7184TT36RUS       1 2,100.00        Ford   Focus
3    X582HE161RUS       2 2,000.00        Ford   Focus
4    92918M178RUS       1 5,700.00        Ford   Focus
..            ...     ...      ...         ...     ...
195  8182XX154RUS       1 9,500.00        Ford   Focus
196   X796TH96RUS       1 2,000.00        Ford   Focus
197  T011MY163RUS       2 4,400.00        Ford   Focus
198   T341CC96RUS       2   800.00  Volkswagen  Passat
199   T119CT96RUS       2 1,400.00        Ford   Focus

[925 rows x 5 columns]

Дополним concat_rows новым столбцом со сгенерированными данными

In [9]:
# Задаем параметры генерации случайных чисел
np.random.seed(21)

◦ создадим серию с названием Year, используя случайные целые числа с 1980 по 2019 год. Надо использовать np.random.seed(21) перед генерацией лет.

Объединим серию с датафреймом

In [10]:
Year = pd.Series([np.random.randint(1980,2019) for i in range(len(concat_rows))])
Year

0      1989
1      1995
2      1984
3      2015
4      2014
       ... 
920    1996
921    2002
922    1996
923    2012
924    1984
Length: 925, dtype: int64

In [17]:
concat_rows['Year'] = Year.astype('int')
fines = concat_rows
fines

CarNumber  Refund    Fines        Make   Model  Year
0    Y163O8161RUS       2 3,200.00        Ford   Focus  1989
1     E432XX77RUS       1 6,500.00      Toyota   Camry  1995
2     7184TT36RUS       1 2,100.00        Ford   Focus  1984
3    X582HE161RUS       2 2,000.00        Ford   Focus  2015
4    92918M178RUS       1 5,700.00        Ford   Focus  2014
..            ...     ...      ...         ...     ...   ...
195  8182XX154RUS       1 9,500.00        Ford   Focus  2012
196   X796TH96RUS       1 2,000.00        Ford   Focus  1998
197  T011MY163RUS       2 4,400.00        Ford   Focus  1983
198   T341CC96RUS       2   800.00  Volkswagen  Passat  1983
199   T119CT96RUS       2 1,400.00        Ford   Focus  1989

[925 rows x 6 columns]

Добавим еще выдуманных данных


Создадим новый фрейм данных с номерами автомобилей и их владельцами. Для этого:

Считаем файл с самыми распространенными фамилиями

In [12]:
surname = pd.read_json('../data/surname.json')

Создадим новую серию с фамилиями (в них не должно быть специальных символов, таких как запятые, скобки и т.д.) Количество должно быть равно количеству уникальных номеров автомобилей. Использовать random_state = 21

Создадим столбец из уникальных номеров. Заодно посмотрим, сколько их для создания списка фамилий.

In [13]:
owner_car = concat_rows.drop_duplicates('CarNumber')['CarNumber']
owner_car = owner_car.reset_index(drop=True)
owner_car

0      Y163O8161RUS
1       E432XX77RUS
2       7184TT36RUS
3      X582HE161RUS
4      92918M178RUS
           ...     
526    O136HO197RUS
527    O22097197RUS
528    M0309X197RUS
529    O673E8197RUS
530    8610T8154RUS
Name: CarNumber, Length: 531, dtype: object

Создадим такой же по длине столбец фамилий.

In [14]:
owner_name = surname['NAME'].sample(n=531, random_state=21, replace=True, ignore_index=True)
owner_name

0      RICHARDSON
1            ROSS
2          MORGAN
3          BAILEY
4           LOPEZ
          ...    
526      CAMPBELL
527          HALL
528         BAKER
529          DIAZ
530        MORGAN
Name: NAME, Length: 531, dtype: object

Создадим датафрейм с 2 столбцами: номер и фамилия

In [15]:
owners = pd.DataFrame(data=owner_car)
owners['SURNAME'] = owner_name
owners

CarNumber     SURNAME
0    Y163O8161RUS  RICHARDSON
1     E432XX77RUS        ROSS
2     7184TT36RUS      MORGAN
3    X582HE161RUS      BAILEY
4    92918M178RUS       LOPEZ
..            ...         ...
526  O136HO197RUS    CAMPBELL
527  O22097197RUS        HALL
528  M0309X197RUS       BAKER
529  O673E8197RUS        DIAZ
530  8610T8154RUS      MORGAN

[531 rows x 2 columns]

Подготовим данные для демонстрации умения объединять таблицы по разным условиям

Добавим еще 5 наблюдений к датафрейму данных о штрафах (придумайте свои собственные идеи)

In [21]:

sample_5 = df.sample(n=5, random_state=30).reset_index(drop=True)
sample_5['CarNumber'] = ['E644TH36RUS', 'E643TH36RUS', 'E642TH36RUS', 'E641TH36RUS', 'E640TH36RUS']
sample_5

CarNumber  Refund    Fines   Make    Model
0  E644TH36RUS       1 8,300.00   Ford    Focus
1  E643TH36RUS       1 8,594.60   Ford    Focus
2  E642TH36RUS       1 1,300.00  Skoda  Octavia
3  E641TH36RUS       1 2,400.00   Ford    Focus
4  E640TH36RUS       1 2,400.00   Ford    Focus

In [22]:
fines = pd.concat([fines, sample_5])
fines

CarNumber  Refund    Fines    Make    Model     Year
0   Y163O8161RUS       2 3,200.00    Ford    Focus 1,989.00
1    E432XX77RUS       1 6,500.00  Toyota    Camry 1,995.00
2    7184TT36RUS       1 2,100.00    Ford    Focus 1,984.00
3   X582HE161RUS       2 2,000.00    Ford    Focus 2,015.00
4   92918M178RUS       1 5,700.00    Ford    Focus 2,014.00
..           ...     ...      ...     ...      ...      ...
0    E644TH36RUS       1 8,300.00    Ford    Focus      NaN
1    E643TH36RUS       1 8,594.60    Ford    Focus      NaN
2    E642TH36RUS       1 1,300.00   Skoda  Octavia      NaN
3    E641TH36RUS       1 2,400.00    Ford    Focus      NaN
4    E640TH36RUS       1 2,400.00    Ford    Focus      NaN

[930 rows x 6 columns]

Удалим из датафрейма владельцев последние 20 наблюдений и добавим 3 новых 

In [23]:
owners_test = owners.drop(owners.tail(20).index)
add3 = owners.sample(n=3, random_state=1)
add3['CarNumber'] = ['E639TH36RUS', 'E638TH36RUS', 'E637TH36RUS']
owners_test = pd.concat([owners_test, add3], ignore_index=True)
owners_test

CarNumber  SURNAME
110  E639TH36RUS  MORALES
289  E638TH36RUS    EVANS
331  E637TH36RUS  BENNETT

In [24]:
owners_test = owners.drop(owners.tail(20).index)
add3 = owners.sample(n=3, random_state=1)
owners_test = pd.concat([owners_test, add3], ignore_index=True)
owners_test

CarNumber     SURNAME
0    Y163O8161RUS  RICHARDSON
1     E432XX77RUS        ROSS
2     7184TT36RUS      MORGAN
3    X582HE161RUS      BAILEY
4    92918M178RUS       LOPEZ
..            ...         ...
509  O50197197RUS      WRIGHT
510  7608EE777RUS        HILL
511  X757HY197RUS     MORALES
512   H219X836RUS       EVANS
513  K2817K154RUS     BENNETT

[514 rows x 2 columns]

Будем соединять эти датафреймы разными способами

Подробно и понятно про агрегирование:

https://sparkbyexamples.com/pandas/pandas-groupby-sum-examples/

∗ новый фрейм данных должен содержать только номера автомобилей, существующие в обоих фреймах данных.

In [25]:
test1 = pd.merge(fines, owners_test, on='CarNumber', how='inner')
test1

CarNumber  Refund     Fines    Make    Model     Year     SURNAME
0    Y163O8161RUS       2  3,200.00    Ford    Focus 1,989.00  RICHARDSON
1    Y163O8161RUS       2  1,600.00    Ford    Focus 1,999.00  RICHARDSON
2     E432XX77RUS       1  6,500.00  Toyota    Camry 1,995.00        ROSS
3     E432XX77RUS       2 13,000.00  Toyota    Camry 1,992.00        ROSS
4     7184TT36RUS       1  2,100.00    Ford    Focus 1,984.00      MORGAN
..            ...     ...       ...     ...      ...      ...         ...
897  E41977152RUS       2  2,400.00    Ford    Focus 2,001.00       BAKER
898  9464EX178RUS       2  2,100.00    Ford    Focus 1,993.00      MARTIN
899  O50197197RUS       2  7,800.00    Ford    Focus 1,986.00      WRIGHT
900  7608EE777RUS       1  4,000.00   Skoda  Octavia 2,013.00        HILL
901  7608EE777RUS       1 34,000.00   Skoda  Octavia 2,011.00        HILL

[902 rows x 7 columns]

∗ новый фрейм данных должен содержать все номера автомобилей, существующие в обоих фреймах данных.

In [26]:
test2 = pd.merge(fines, owners_test, on='CarNumber', how='outer')
test2

CarNumber  Refund     Fines    Make    Model     Year     SURNAME
0    Y163O8161RUS       2  3,200.00    Ford    Focus 1,989.00  RICHARDSON
1    Y163O8161RUS       2  1,600.00    Ford    Focus 1,999.00  RICHARDSON
2     E432XX77RUS       1  6,500.00  Toyota    Camry 1,995.00        ROSS
3     E432XX77RUS       2 13,000.00  Toyota    Camry 1,992.00        ROSS
4     7184TT36RUS       1  2,100.00    Ford    Focus 1,984.00      MORGAN
..            ...     ...       ...     ...      ...      ...         ...
928   E644TH36RUS       1  8,300.00    Ford    Focus      NaN         NaN
929   E643TH36RUS       1  8,594.60    Ford    Focus      NaN         NaN
930   E642TH36RUS       1  1,300.00   Skoda  Octavia      NaN         NaN
931   E641TH36RUS       1  2,400.00    Ford    Focus      NaN         NaN
932   E640TH36RUS       1  2,400.00    Ford    Focus      NaN         NaN

[933 rows x 7 columns]

∗ новый фрейм данных должен содержать только номера автомобилей из фрейма данных штрафов

In [28]:
test3 = pd.merge(fines, owners_test, on='CarNumber', how='left')
test3

CarNumber  Refund    Fines    Make    Model     Year     SURNAME
0    Y163O8161RUS       2 3,200.00    Ford    Focus 1,989.00  RICHARDSON
1     E432XX77RUS       1 6,500.00  Toyota    Camry 1,995.00        ROSS
2     7184TT36RUS       1 2,100.00    Ford    Focus 1,984.00      MORGAN
3    X582HE161RUS       2 2,000.00    Ford    Focus 2,015.00      BAILEY
4    92918M178RUS       1 5,700.00    Ford    Focus 2,014.00       LOPEZ
..            ...     ...      ...     ...      ...      ...         ...
928   E644TH36RUS       1 8,300.00    Ford    Focus      NaN         NaN
929   E643TH36RUS       1 8,594.60    Ford    Focus      NaN         NaN
930   E642TH36RUS       1 1,300.00   Skoda  Octavia      NaN         NaN
931   E641TH36RUS       1 2,400.00    Ford    Focus      NaN         NaN
932   E640TH36RUS       1 2,400.00    Ford    Focus      NaN         NaN

[933 rows x 7 columns]

∗ новый фрейм данных должен содержать только номера автомобилей из
фрейма данных владельцев

In [29]:
test4 = pd.merge(fines, owners_test, on='CarNumber', how='right')
test4

CarNumber  Refund     Fines    Make    Model     Year     SURNAME
0    Y163O8161RUS       2  3,200.00    Ford    Focus 1,989.00  RICHARDSON
1    Y163O8161RUS       2  1,600.00    Ford    Focus 1,999.00  RICHARDSON
2     E432XX77RUS       1  6,500.00  Toyota    Camry 1,995.00        ROSS
3     E432XX77RUS       2 13,000.00  Toyota    Camry 1,992.00        ROSS
4     7184TT36RUS       1  2,100.00    Ford    Focus 1,984.00      MORGAN
..            ...     ...       ...     ...      ...      ...         ...
897  7608EE777RUS       1  4,000.00   Skoda  Octavia 2,013.00        HILL
898  7608EE777RUS       1 34,000.00   Skoda  Octavia 2,011.00        HILL
899  X757HY197RUS       2  6,800.00    Ford    Focus 2,010.00     MORALES
900   H219X836RUS       1  1,600.00    Ford    Focus 2,000.00       EVANS
901  K2817K154RUS       1  4,500.00    Ford    Focus 1,991.00     BENNETT

[902 rows x 7 columns]

Cоздадим сводную таблицу по заданию

In [62]:
pd.pivot_table(car_test,
               columns='Year',
               values='Fines',
               index=['Make', 'Model'],
               aggfunc={'Fines': np.sum})

Year                    1980       1981       1982       1983       1984  \
Make       Model                                                           
Ford       Focus   70,594.60 310,878.40 138,800.00 145,094.60 107,900.00   
           Mondeo        NaN        NaN  46,200.00        NaN        NaN   
Skoda      Octavia  8,594.60   1,900.00  43,194.60  16,300.00     300.00   
Toyota     Camry   12,000.00        NaN        NaN        NaN   1,000.00   
           Corolla       NaN   6,800.00        NaN  12,800.00   2,500.00   
Volkswagen Golf    32,200.00   8,594.60   5,000.00     200.00        NaN   
           Jetta         NaN   1,000.00        NaN        NaN        NaN   
           Passat     900.00   4,100.00        NaN   1,900.00   8,594.60   
           Touareg       NaN        NaN        NaN        NaN        NaN   

Year                     1985      1986      1987      1988       1989  ...  \
Make       Model                                                        ...   
Ford       Focus   336,389.20 62,200.00 71,594.60 60,278.40 182,689.20  ...   
           Mondeo         NaN       NaN       NaN       NaN        NaN  ...   
Skoda      Octavia   9,994.60       NaN  2,000.00  5,100.00   8,594.60  ...   
Toyota     Camry          NaN 24,400.00       NaN       NaN     800.00  ...   
           Corolla        NaN       NaN 54,800.00       NaN   7,800.00  ...   
Volkswagen Golf    168,000.00       NaN       NaN       NaN     300.00  ...   
           Jetta     9,000.00       NaN       NaN 46,000.00        NaN  ...   
           Passat         NaN 16,000.00  2,000.00  8,594.60        NaN  ...   
           Touareg        NaN       NaN       NaN       NaN        NaN  ...   

Year                     2009       2010       2011       2012       2013  \
Make       Model                                                            
Ford       Focus   200,094.60 241,600.00 101,794.60 182,189.20 249,389.20   
           Mondeo         NaN        NaN        NaN        NaN  41,100.00   
Skoda      Octavia        NaN   2,500.00  37,000.00   1,700.00  11,800.00   
Toyota     Camry          NaN  22,400.00        NaN   7,500.00        NaN   
           Corolla   8,594.60   6,000.00        NaN        NaN   3,600.00   
Volkswagen Golf           NaN        NaN        NaN        NaN        NaN   
           Jetta          NaN        NaN        NaN        NaN   1,100.00   
           Passat         NaN   9,500.00        NaN        NaN   1,600.00   
           Touareg   5,800.00        NaN        NaN        NaN        NaN   

Year                    2014       2015       2016       2017       2018  
Make       Model                                                          
Ford       Focus   80,178.40 355,694.60 109,983.80 142,594.60 103,200.00  
           Mondeo        NaN        NaN        NaN   8,600.00        NaN  
Skoda      Octavia  7,000.00  16,394.60  45,300.00   2,400.00 153,200.00  
Toyota     Camry         NaN        NaN        NaN        NaN        NaN  
           Corolla       NaN        NaN        NaN        NaN        NaN  
Volkswagen Golf    43,600.00        NaN        NaN        NaN        NaN  
           Jetta         NaN        NaN        NaN        NaN        NaN  
           Passat   6,700.00        NaN        NaN        NaN        NaN  
           Touareg       NaN        NaN        NaN        NaN        NaN  

[9 rows x 39 columns]

Сохраним фреймы данных о штрафах и владельцах в CSV-файлах без индекса

In [63]:
car_test.to_csv('../data/fines.csv', index=False)
owners.to_csv('../data/owners.csv', index=False)

In [64]:
len(fines)

930